In [1]:
import blackhc.notebook

Appended /home/blackhc/PycharmProjects/dumbo/src to paths
Switched to directory /home/blackhc/PycharmProjects/dumbo
%load_ext autoreload
%autoreload 2


In [2]:
import dumbo
import dumbo.support.ipython
import dumbo.support.torch

In [3]:
import torch

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [19]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [56]:
%%dumbo mnist_model
global model
use_cuda = torch.cuda.is_available()
seed = 0
epochs = 1

torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

batch_size = 256
test_batch_size = 1024
lr = 0.001
gamma=0.7

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, 50)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303637
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.272697
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.249982
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.215464
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.174778

Test set: Average loss: 2.1155, Accuracy: 4883/10000 (49%)



In [28]:
dumbo.get_cached_value_identities()

dict_keys([ValueCellResultIdentity(cell=CellIdentity(qualified_name='mnist_model'), key='model')])

In [33]:
from pprint import pprint

In [34]:
pprint(dumbo.main.dumbo.online_cache.vid_to_value)

{ValueCellResultIdentity(cell=CellIdentity(qualified_name='mnist_model'), key='model'): StoredResult(value=Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25)
  (dropout2): Dropout2d(p=0.5)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
), fingerprint=CellResultFingerprint(cell=CellFingerprint(cell_code_fingerprint=DeepFunctionFingerprint(fingerprint=b'\x18\x88\x0c\x1f\x17\xe0\x97\xb2WU\x1dh)5A=', func_calls=frozenset({(('Net',), FunctionFingerprint(fingerprint=b'\x9f\xbf\x0e3\xd9\xf0Oi\x053\xf3Mx\xebe\xcf')), (('test',), DeepFunctionFingerprint(fingerprint=b'\x0b\xdd__\x93\xdf\xd1\xfe\xb46\x06\xb2,\xa1 j', func_calls=frozenset())), (('train',), DeepFunctionFingerprint(fingerprint=b'\xe0+\xf7\x9ay\x9f\xd2\x00\xd6\xb3\x80\xd9\xacHl\xc3', func_calls=frozenset()))})), globals_load=frozenset({(('StepLR',), (Va

In [57]:
dumbo.is_stale(model, depth=-1)

True

In [46]:
dumbo.main.dumbo._get_vid(model)

ValueCellResultIdentity(cell=CellIdentity(qualified_name='mnist_model'), key='model')

In [49]:
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25)
  (dropout2): Dropout2d(p=0.5)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)